In [1]:
# import os
# from pathlib import Path
# from google.colab import drive

# PATH = "/content/drive"
# drive.mount(PATH)

# folder_path = Path('MyDrive/models/abs_bert')

In [2]:
# !pip install --quiet transformers datasets
# !pip install --quiet tensorboard
# tensorboard --logdir=runs

In [1]:
import torch
device = 'cuda:pick_a_device' if torch.cuda.is_available() else 'cpu'
torch.cuda.is_available()

False

In [ ]:
import os
import datetime
from pathlib import Path
import json
from tqdm import tqdm

import torch.optim.lr_scheduler as lr_scheduler

# import datasets
# from datasets import load_dataset
from transformers import AutoTokenizer

# import utils.load_corpus as load_corpus

from absolute_bert.models.absolute_bert import models, config
from absolute_bert import losses

import importlib

ModuleNotFoundError: No module named 'absolute_bert.pick_a_model'

In [5]:
# colab
# folder_path = Path(folder_path)
# os.makedirs(PATH/folder_path, exist_ok=True)

# 59
PATH = "data"
folder_path = Path(f"limanet")
os.makedirs(PATH/folder_path, exist_ok=True)

In [6]:
tokenizer_type = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
text = ["I love bird.", "Hi, I'm Bob."]
tokenized = tokenizer(text, return_tensors='pt', padding=True)
tokenized['input_ids']

tensor([[ 101, 1045, 2293, 4743, 1012,  102,    0,    0,    0],
        [ 101, 7632, 1010, 1045, 1005, 1049, 3960, 1012,  102]])

In [ ]:
%%time
corpus_name = 'bookcorpus'
# corpus_name = 'msmarco'
# corpus_name = 'wiki'

batch_size = 128
masking_probability = .15
max_length = 256
num_chunks = 1
shuffle = True
# cache_dir = PATH/Path('MyDrive/data')
cache_dir='~/data1-0756727/cache/huggingface'

corpus = load_corpus.CorpusForTransformer(
    corpus_name,
    tokenizer,
    cache_dir,
    batch_size=batch_size,
    max_length=max_length,
    num_chunks=num_chunks, 
    shuffle=False
)

train_loader = corpus.loader()
n = next(iter(train_loader))
# print(tokenizer.decode(n['data']))
{key: n[key].shape for key in n}

/home/adam/miniconda3/envs/lima/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for bookcorpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bookcorpus
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


CPU times: user 17.8 s, sys: 7.43 s, total: 25.2 s
Wall time: 27.4 s


{'input_ids': torch.Size([128, 50]),
 'token_type_ids': torch.Size([128, 50]),
 'attention_mask': torch.Size([128, 50]),
 'labels': torch.Size([128, 50])}

In [ ]:
learning_rate = 0.01  # for cross entropy
learning_rate = 4e-5  # for fine tuning

model_config = config.AbsoluteBertConfig(
	vocab_size = tokenizer.vocab_size,
	depth = 8,
	num_heads = 8,
	dim = 512,
	# hidden_dim = 32,
	# time_dim = 64,
	# embedding_initialize_method = 'rand',
  	# attention_type = Absolute_global_attention,
)

loss_params = {
  'sampling_word_size': 100,
}

model = models.AbsoluteBertForMaskedLM(model_config).to(device)
using_loss = losses.SampledSoftmaxCrossEntropy(model, num_sampling=loss_params['sampling_word_size'])

# model = torch.load(PATH/folder_path/subpath/f'batch_{batch_num}-model.pt', map_location=device)


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=1e-5)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 2500, eta_min=1e-6)
optimizer


AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-05
    foreach: None
    fused: None
    initial_lr: 4e-05
    lr: 4e-05
    maximize: False
    weight_decay: 0.01
)

In [ ]:
# from typing import Any
# from pathlib import Path

# class TrainDumper:
# 	def __init__(self, model_config: Any, saving_dir: Path, timezone:int=8) -> None:
# 		self.model_config = model_config

# 	def dump(self) -> None:
# 		timezone = datetime.timezone(datetime.timedelta(hours=8))
# 		time_str = datetime.datetime.now(tz=timezone).strftime('%Y%m%d.%H:%M:%S')

In [ ]:
from dataclasses import 

time_string = datetime.datetime.now(tz=datetime.timezone(datetime.timedelta(hours=8))).strftime('%Y%m%d.%H:%M:%S')
subfolder_path = Path(time_string)

os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

with open(PATH/folder_path/subfolder_path/f'parameters.json', 'w') as f:
  f.write(json.dumps({
    'tokenizer_type': tokenizer_type,
    'model': str(model),
    'model_params': str(model_params),
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'masking_probability': masking_probability,
    'shuffle': shuffle,
    'loss_type': str(using_loss),
    'loss_params': loss_params,
  }))
time_string

'20240711.15:36:49'

In [11]:
from torch.utils.tensorboard import SummaryWriter
from utils import benchmark

benchmark = importlib.reload(benchmark)

# Writer will output to ./runs/ directory by default
writer = SummaryWriter(log_dir=PATH/folder_path/subfolder_path)
benchmark_writer = benchmark.Benchmarking(writer, tokenizer)

  0%|          | 0/5183 [00:00<?, ?it/s]

In [12]:
num_epochs = 2
save_every_n_batches = 5000
clip_loss = 50

multiloss = 0


print(time_string)

# os.makedirs(PATH/folder_path/subfolder_path, exist_ok=True)

model.train() 
global_step = 0

loss_history = []
for epoch_num, epoch in enumerate(range(num_epochs)):
  bar = tqdm(train_loader)

  for batch_num, batch in enumerate(bar):

    optimizer.zero_grad()

    batch = {key: batch[key].to(device) for key in batch}
    predicts, targets = model(**batch)

    # predicts = predicts @ model.word_embeddings().T + model.bias
    # predicts = predicts.view(-1, tokenizer.vocab_size)
    # targets = targets.view(-1)

    if multiloss:
      losses = using_loss(predicts, targets)
      loss = sum(losses.values())
    else:
      loss = using_loss(predicts, targets)

    if clip_loss is not None:
      final_loss = torch.clip(loss, max=clip_loss)
    else:
      final_loss = loss

    loss_dict = {
      'loss': f"{loss.item(): .6f}",
      'final_loss': f"{final_loss.item(): .6f}",
    }

    if global_step == 0:
      writer.add_scalar('Loss/loss', loss, global_step)
      benchmark_writer.predict_and_write(model.word_embeddings().detach().cpu(), 0)
    
    if multiloss:
      loss_dict |= {key: f"{losses[key].item(): .6f}" for key in losses}

    bar.set_postfix(loss_dict)

    # with torch.autograd.detect_anomaly(True):
      # loss.backward()
      # torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=10, norm_type=2)

    final_loss.backward()
    optimizer.step()
    global_step += 1
    
    if (batch_num % 100 == 0):
      # torch.cuda.empty_cache()
      loss_history.append([epoch_num, batch_num, loss_dict])
      # print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, batch {batch_num}")

    if (global_step % 100 == 0):
      writer.add_scalar('Loss/loss', loss, global_step)

    if (batch_num % 1000 == 999):
      scheduler.step()

    if (batch_num % save_every_n_batches == 0):
      prefix = f"epoch_{epoch_num}-batch_{batch_num}-"
      torch.save(model, PATH/folder_path/subfolder_path/f'{prefix}-model.pt')

      with open(PATH/folder_path/subfolder_path/f'epoch_{epoch_num}-history.json', 'w') as f:
        f.write(json.dumps(loss_history))

      # torch.cuda.empty_cache()

      benchmark_writer.predict_and_write(model.word_embeddings().detach().cpu(), global_step)

    if (batch_num in [500, 1000, 1500, 2000, 2500, 7500, 12500, 17500]):
      prefix = f"epoch_{epoch_num}-batch_{batch_num}"
      torch.save(model, PATH/folder_path/subfolder_path/f'{prefix}-model.pt')

      with open(PATH/folder_path/subfolder_path/f'epoch_{epoch_num}-history.json', 'w') as f:
        f.write(json.dumps(loss_history))

      benchmark_writer.predict_and_write(model.word_embeddings().detach().cpu(), global_step)
          
    # del batch, loss

20240711.15:36:49


  0%|          | 879/578159 [01:55<21:05:11,  7.60it/s, loss=7.716620, final_loss=7.716620]   


KeyboardInterrupt: 